<a href="https://colab.research.google.com/github/yang1325/AI_selfpraticenote/blob/main/tensorflow_covid/tensorflow_Coviddata_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#載入所需模組
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19
from tensorflow import keras
import numpy as np
import pandas as pd
from numpy import load
from numpy import save
import random
import os
import gc
from datetime import datetime, timezone, timedelta
from time import sleep
from tqdm.notebook import tqdm 
tz = timezone(timedelta(hours=+8)) 

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/My Drive/splited_data"
checkpoint_path = "/content/gdrive/My Drive/trained/model4"
checkpoint_dir = os.path.dirname(checkpoint_path)

#cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#        save_weights_only=True
#        ,verbose=1)

#cnn_model.compile(optimizer="Adam",
#              loss=tf.keras.losses.categorical_crossentropy,
#              metrics=['accuracy'])

cnn_model = tf.keras.models.load_model(checkpoint_path)
#cnn_model.load_weights(checkpoint_path)

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
def loss(model, x, y, training):
  y_ = model(x, training=training)
  return y_ , loss_object(y_true=y, y_pred=y_)
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    y_ ,loss_value = loss(model, inputs, targets, training=True)
  return y_ , loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
batch = 30
fmt = '(%H:%M:%S)'
for k in range(100):
  ran = np.array(random.sample(range(0,90),90))
  for j in range(4):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.CategoricalAccuracy()
    if(j < 3):
      end = j*30+30
    else:
      ran = np.array(range(0,100))
      end = 100
    data = load(path + "/%d"%(ran[j*30]) + ".npy")
    label = load(path + "/%dy"%(ran[j*30]) + ".npy")
    for pic in range(j*30 + 1,end):
      tem = load(path + "/%d"%(ran[pic]) + ".npy")
      temy = load(path + "/%dy"%(ran[pic]) + ".npy")
      data = np.concatenate([data,tem],axis = 0)
      label = np.concatenate([label,temy],axis = 0)
      del tem
      del temy
      gc.collect()
    n = np.shape(data)[0]
    rand = np.array(random.sample(range(0,n),n))
    gc.collect()
    z = np.apply_along_axis(np.argmax,1,label)
    t = tf.convert_to_tensor(data)
    del data
    if(j<3):
      #cnn_model.fit(data, label, epochs=1,batch_size = batch,callbacks=[cp_callback])
      zoned_time1 = datetime.today().astimezone(tz)
      with tqdm(total=n) as pbar:
        pbar.set_description("%s%s:"%(("epoch:%d.%d,"%(k,j))
        ,zoned_time1.strftime(fmt)))
        for i in range(0,n,batch):
          image =  np.zeros((batch,299,299),dtype=int)
          y = np.zeros((batch,4),dtype=int)
          for l in range(batch):
            image[l,:,:] = t[rand[i+l]:rand[i+l]+1]
            #image[:,:,:,1] = t[i:(i+batch)]
            #image[:,:,:,2] = t[i:(i+batch)]
            y[l,:] = label[rand[i+l]:rand[i+l]+1]
          y_ , loss_value, grads = grad(cnn_model, image, y)
          optimizer.apply_gradients(zip(grads, cnn_model.trainable_variables))
          epoch_loss_avg.update_state(loss_value)
          for l in range(batch):
            epoch_accuracy.update_state(y[l:l+1],y_[l:(l+1)])
          pbar.update(batch)
          pbar.set_postfix(desc = "Loss: {:.3f}, Accuracy: {:.3%}".format(
                 epoch_loss_avg.result(),
                 epoch_accuracy.result()))
      cnn_model.save(checkpoint_path)
    else:
      #cnn_model.evaluate(t[i:(i+1)], z[i:(i+1)])
      zoned_time1 = datetime.today().astimezone(tz)
      with tqdm(total=n) as pbar:
        pbar.set_description("%s%s:"%(("epoch:%d,validation,"%(k))
        ,zoned_time1.strftime(fmt)))
        for i in range(n):
          image =  np.zeros((1,299,299,1))
          image[:,:,:,0] = t[i:(i+1)]
          #image[:,:,:,1] = t[i:(i+1)]
          #image[:,:,:,2] = t[i:(i+1)]
          y_ , loss_value, grads = grad(cnn_model, image, label[i:(i+1)])
          epoch_loss_avg.update_state(loss_value)
          epoch_accuracy.update_state(label[i:(i+1)],y_)
          pbar.update(1)
          pbar.set_postfix(desc = "Loss: {:.3f}, Accuracy: {:.3%}".format(
                 epoch_loss_avg.result(),
                 epoch_accuracy.result()))
    del t
    gc.collect()

  0%|          | 0/6330 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/trained/model4/assets


  0%|          | 0/6330 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/trained/model4/assets


  0%|          | 0/6330 [00:00<?, ?it/s]

...

  0%|          | 0/6330 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/trained/model4/assets


  0%|          | 0/6330 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/trained/model4/assets


  0%|          | 0/6330 [00:00<?, ?it/s]